In [1]:
from utils import QualityIndexStore,get_day_df
import requests
import pandas as pd
import datetime as dt


def get_general_data(standort):
    url = f"https://www.oekumenische-energiegenossenschaft.de/datenlogger/{standort}/visualisierung/base_vars.js"
    response  = requests.get(url)
    key_value_list = [x[4:].split("=") for x in response.text.split("\n") if x.startswith("var ")]
    return pd.DataFrame(key_value_list, columns=["key", "value"]).set_index("key").T

standorte = ["badboll","esslingen","geislingen","holzgerlingen","hospitalhof","karlsruhe","mettingen","muensingen","tuebingen","waiblingen"]
final_df = []
for s in standorte:
    temp_general = get_general_data(s)
    quality = QualityIndexStore(s, days_back=30).get_data().mean().reset_index()
    quality = quality[0].groupby(quality["index"].str.split("_").str[0]).mean().reset_index().T
    
    column_names = quality.iloc[0]
    quality = quality[1:]
    quality.columns = column_names
    date = dt.datetime.now() - dt.timedelta(days=1)
    
    last_day = get_day_df(s,date.strftime("%y%m%d")) 
    
    if len(column_names)==0:
        res=temp_general.to_dict()
        res["s"] = s
        res["Standort"] = temp_general["HPStandort"].iloc[0].strip().strip('"')
        res["Max. Leistung"] = f"{int(temp_general['AnlagenKWP'].iloc[0])/1000} kWp"
        res["Wechselrichter"] = "--"
        res["letzter Tag"] = []
        res["Datenqualität"] = 0
        
        final_df.append(res)
    else:
        for wr in quality.columns:
            res=temp_general.to_dict()
            res["s"] = s
            res["Standort"] = temp_general["HPStandort"].iloc[0].strip().strip('"')
            res["Max. Leistung"] = f"{int(temp_general['AnlagenKWP'].iloc[0])/1000} kWp" #temp_general["HPLeistung"].iloc[0].strip('" ')
            
            res["Wechselrichter"] = wr
            res["letzter Tag"] = last_day[wr+"_P"].values if isinstance(last_day,pd.DataFrame) else []
            res["Datenqualität"] = quality[wr].iloc[0]
        
            final_df.append(res)
            
df = pd.DataFrame(final_df).fillna(0)


OSError: [Errno 30] Read-only file system: '/app'

In [39]:
import requests
import pandas as pd
import datetime as dt
from utils import QualityIndexStore, get_day_df

def get_general_data(standort: str) -> pd.DataFrame:
    """Lädt die Grunddaten eines Standorts als DataFrame"""
    url = f"https://www.oekumenische-energiegenossenschaft.de/datenlogger/{standort}/visualisierung/base_vars.js"
    response = requests.get(url)
    key_value_list = [
        line[4:].split("=")
        for line in response.text.split("\n")
        if line.startswith("var ")
    ]
    df = pd.DataFrame(key_value_list, columns=["key", "value"]).set_index("key").T
    return df


def process_quality_data(standort: str) -> pd.DataFrame:
    """Aggregiert Qualitätsdaten eines Standorts über 30 Tage"""
    quality = QualityIndexStore(standort, days_back=30).get_data().mean().reset_index()
    if quality.empty:
        return pd.DataFrame()
    
    quality_grouped = (
        quality[0]
        .groupby(quality["index"].str.split("_").str[0])
        .mean()
        .reset_index()
        .T
    )
    
    column_names = quality_grouped.iloc[0]
    data = quality_grouped[1:]
    data.columns = column_names
    return data


def build_standort_entry(standort: str, general_df: pd.DataFrame, quality_df: pd.DataFrame, last_day_df: pd.DataFrame) -> list[dict]:
    """Erzeugt eine Liste von Dictionaries pro Wechselrichter für einen Standort"""
    eintraege = []
    
    max_leistung_kwp = f"{int(general_df['AnlagenKWP'].iloc[0]) / 1000} kWp"
    standort_name = general_df["HPStandort"].iloc[0].strip().strip('"')
    
    if quality_df.empty:
        # Keine Qualitätsdaten vorhanden
        eintraege.append({
            "s": standort,
            "Standort": standort_name,
            "Max. Leistung": max_leistung_kwp,
            "Wechselrichter": "--",
            "letzter Tag": [],
            "Datenqualität": 0,
            **general_df.to_dict(orient="records")[0]
        })
    else:
        for wr in quality_df.columns:
            eintraege.append({
                "s": standort,
                "Standort": standort_name,
                "Max. Leistung": max_leistung_kwp,
                "Wechselrichter": wr,
                "letzter Tag": last_day_df.get(wr + "_P", pd.Series([])).values.tolist() if isinstance(last_day_df, pd.DataFrame) else [],
                "Datenqualität": quality_df[wr].iloc[0],
                **general_df.to_dict(orient="records")[0]
            })
    
    return eintraege


standorte = [
    "badboll", "esslingen", "geislingen", "holzgerlingen", "hospitalhof",
    "karlsruhe", "mettingen", "muensingen", "tuebingen", "waiblingen"
]

gesamt_df = []
gestriger_tag = dt.datetime.now() - dt.timedelta(days=1)
gestriges_datum = gestriger_tag.strftime("%y%m%d")

for standort in standorte:
    general_df = get_general_data(standort)
    quality_df = process_quality_data(standort)
    last_day_df = get_day_df(standort, gestriges_datum)
    
    eintraege = build_standort_entry(standort, general_df, quality_df, last_day_df)
    gesamt_df.extend(eintraege)

df = pd.DataFrame(gesamt_df).fillna(0)

df

,s,Standort,Max. Leistung,Wechselrichter,letzter Tag,Datenqualität,Boot,AnlagenKWP,time_start,time_end,...,DSTMode,Dezimalseparator,WeightUnit,DirectMarketing,AdamAvailable,netProfile,pmControlType,pmReductionOnSerialType,windinverters,co2factor
0,badboll,Ev. Akademie Bad Boll,32.13 kWp,WR1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.000000,99\r,32130\r,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,1\r,2\r,0\r,700\r
1,badboll,Ev. Akademie Bad Boll,32.13 kWp,WR2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.968132,99\r,32130\r,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,1\r,2\r,0\r,700\r
2,badboll,Ev. Akademie Bad Boll,32.13 kWp,WR3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.968131,99\r,32130\r,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,1\r,2\r,0\r,700\r
3,esslingen,Esslingen Waldheim,45.0 kWp,WR1,[],0.000000,99,45000,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)","new Array(17,18,20,21,21,22,22,21,20,19,17,17)",...,"""1""",""",""","""KG""",false,0,0,0,0,0,460
4,esslingen,Esslingen Waldheim,45.0 kWp,WR2,[],0.000000,99,45000,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)","new Array(17,18,20,21,21,22,22,21,20,19,17,17)",...,"""1""",""",""","""KG""",false,0,0,0,0,0,460
5,esslingen,Esslingen Waldheim,45.0 kWp,WR3,[],0.000000,99,45000,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)","new Array(17,18,20,21,21,22,22,21,20,19,17,17)",...,"""1""",""",""","""KG""",false,0,0,0,0,0,460
6,geislingen,Ev. Pfarrhaus Geislingen,5.7 kWp,WR1,[],0.000000,99\r,5700\r,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r",0,0,0,0,0,0,0,0
7,holzgerlingen,Holzgerlingen,17.64 kWp,WR1,"[0, 0, 3, 16, 29, 40, 54, 69, 83, 98, 111, 126...",0.995684,99\r,17640\r,"new Array(8,7,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,0,0,0\r,0
8,holzgerlingen,Holzgerlingen,17.64 kWp,WR2,"[0, 0, 7, 19, 31, 43, 57, 73, 87, 102, 116, 13...",0.996827,99\r,17640\r,"new Array(8,7,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,0,0,0\r,0
9,hospitalhof,Hospitalhof Stuttgart,36.21 kWp,WR1,[],0.000000,99\r,36210\r,"new Array(8,7,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,0\r,0\r,0\r,700\r


In [3]:
import os
import pickle
import pandas as pd
import datetime as dt
import requests
from utils import QualityIndexStore, get_day_df


class OverviewDatenManager:
    def __init__(self, standorte, pickle_path="standorte_data.pkl"):
        self.standorte = standorte
        self.pickle_path = pickle_path
        self.df = self._load_or_initialize_df()

    def _load_or_initialize_df(self):
        if os.path.exists(self.pickle_path):
            with open(self.pickle_path, "rb") as f:
                return pickle.load(f)
        else:
            return pd.DataFrame()

    def _save_df(self):
        with open(self.pickle_path, "wb") as f:
            pickle.dump(self.df, f)

    def _get_general_data(self, standort: str) -> pd.DataFrame:
        url = f"https://www.oekumenische-energiegenossenschaft.de/datenlogger/{standort}/visualisierung/base_vars.js"
        response = requests.get(url)
        key_value_list = [
            line[4:].split("=")
            for line in response.text.split("\n")
            if line.startswith("var ")
        ]
        df = pd.DataFrame(key_value_list, columns=["key", "value"]).set_index("key").T
        return df

    def _process_quality_data(self, standort: str) -> pd.DataFrame:
        quality = QualityIndexStore(standort, days_back=30).get_data().mean().reset_index()
        if quality.empty:
            return pd.DataFrame()
        quality_grouped = (
            quality[0]
            .groupby(quality["index"].str.split("_").str[0])
            .mean()
            .reset_index()
            .T
        )
        column_names = quality_grouped.iloc[0]
        data = quality_grouped[1:]
        data.columns = column_names
        return data

    def _get_last_day_data(self, standort: str, date: str) -> pd.DataFrame:
        return get_day_df(standort, date)

    def _build_entries(self, standort: str, general_df: pd.DataFrame, quality_df: pd.DataFrame, last_day_df: pd.DataFrame) -> list[dict]:
        eintraege = []
        max_leistung_kwp = f"{int(general_df['AnlagenKWP'].iloc[0]) / 1000} kWp"
        standort_name = general_df["HPStandort"].iloc[0].strip().strip('"')

        if quality_df.empty:
            eintraege.append({
                "s": standort,
                "Standort": standort_name,
                "Max. Leistung": max_leistung_kwp,
                "Wechselrichter": "--",
                "letzter Tag": [],
                "Datenqualität": 0,
                **general_df.to_dict(orient="records")[0]
            })
        else:
            for wr in quality_df.columns:
                eintraege.append({
                    "s": standort,
                    "Standort": standort_name,
                    "Max. Leistung": max_leistung_kwp,
                    "Wechselrichter": wr,
                    "letzter Tag": last_day_df.get(wr + "_P", pd.Series([])).values.tolist()
                        if isinstance(last_day_df, pd.DataFrame) else [],
                    "Datenqualität": quality_df[wr].iloc[0],
                    **general_df.to_dict(orient="records")[0]
                })
        return eintraege

    def _build_df_from_scratch(self):
        """Hilfsfunktion: erstellt das gesamte DataFrame frisch"""
        all_entries = []
        date_str = (dt.datetime.now() - dt.timedelta(days=1)).strftime("%y%m%d")

        for standort in self.standorte:
            general_df = self._get_general_data(standort)
            quality_df = self._process_quality_data(standort)
            last_day_df = self._get_last_day_data(standort, date_str)
            entries = self._build_entries(standort, general_df, quality_df, last_day_df)
            all_entries.extend(entries)

        return pd.DataFrame(all_entries).fillna(0)

    def update_quality_only(self):
        """Aktualisiert nur die Qualitätsdaten"""
        if self.df.empty:
            self.df = self._build_df_from_scratch()

        for idx, row in self.df.iterrows():
            standort = row["s"]
            wr = row["Wechselrichter"]
            if wr == "--":
                continue
            quality_df = self._process_quality_data(standort)
            if wr in quality_df.columns:
                self.df.at[idx, "Datenqualität"] = quality_df[wr].iloc[0]

        self._save_df()

    def update_last_day_only(self):
        """Aktualisiert nur die Daten des letzten Tages"""
        if self.df.empty:
            self.df = self._build_df_from_scratch()

        date_str = (dt.datetime.now() - dt.timedelta(days=1)).strftime("%y%m%d")
        for idx, row in self.df.iterrows():
            standort = row["s"]
            wr = row["Wechselrichter"]
            if wr == "--":
                continue
            last_day_df = self._get_last_day_data(standort, date_str)
            if isinstance(last_day_df, pd.DataFrame) and wr + "_P" in last_day_df.columns:
                self.df.at[idx, "letzter Tag"] = last_day_df[wr + "_P"].values.tolist()

        self._save_df()

    def get_dataframe(self) -> pd.DataFrame:
        """Lädt das DataFrame oder erzeugt es, falls es noch nicht existiert"""
        if self.df.empty:
            # 👉 automatisch initial aufbauen, wenn leer
            self.df = self._build_df_from_scratch()
            self._save_df()

        return self.df.copy()

In [4]:
# Initialisierung
manager = OverviewDatenManager(standorte=[
    "badboll", "esslingen", "geislingen", "holzgerlingen", "hospitalhof",
    "karlsruhe", "mettingen", "muensingen", "tuebingen", "waiblingen"
])

# Erster Aufbau (langsam)
#manager.initial_build_and_save()

# Später im Code:
df = manager.get_dataframe()  # lädt aus Pickle

# Asynchron aufrufen möglich:
# (z. B. in einem Thread, einer Task oder externen CLI)
manager.update_quality_only()
manager.update_last_day_only()

In [52]:
manager.update_last_day_only()

In [ ]:
manager = StandortDatenManager(standorte=[
    "badboll", "esslingen", "geislingen", "holzgerlingen", "hospitalhof",
    "karlsruhe", "mettingen", "muensingen", "tuebingen", "waiblingen"
])

manager.get_dataframe()  

,s,Standort,Max. Leistung,Wechselrichter,letzter Tag,Datenqualität,Boot,AnlagenKWP,time_start,time_end,...,DSTMode,Dezimalseparator,WeightUnit,DirectMarketing,AdamAvailable,netProfile,pmControlType,pmReductionOnSerialType,windinverters,co2factor
0,badboll,Ev. Akademie Bad Boll,32.13 kWp,WR1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,99\r,32130\r,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,1\r,2\r,0\r,700\r
1,badboll,Ev. Akademie Bad Boll,32.13 kWp,WR2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.968132,99\r,32130\r,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,1\r,2\r,0\r,700\r
2,badboll,Ev. Akademie Bad Boll,32.13 kWp,WR3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.968131,99\r,32130\r,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,1\r,2\r,0\r,700\r
3,esslingen,Esslingen Waldheim,45.0 kWp,WR1,[],NaN,99,45000,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)","new Array(17,18,20,21,21,22,22,21,20,19,17,17)",...,"""1""",""",""","""KG""",false,0,0,0,0,0,460
4,esslingen,Esslingen Waldheim,45.0 kWp,WR2,[],NaN,99,45000,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)","new Array(17,18,20,21,21,22,22,21,20,19,17,17)",...,"""1""",""",""","""KG""",false,0,0,0,0,0,460
5,esslingen,Esslingen Waldheim,45.0 kWp,WR3,[],NaN,99,45000,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)","new Array(17,18,20,21,21,22,22,21,20,19,17,17)",...,"""1""",""",""","""KG""",false,0,0,0,0,0,460
6,geislingen,Ev. Pfarrhaus Geislingen,5.7 kWp,WR1,[],NaN,99\r,5700\r,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r",0,0,0,0,0,0,0,0
7,holzgerlingen,Holzgerlingen,17.64 kWp,WR1,"[0, 0, 3, 16, 29, 40, 54, 69, 83, 98, 111, 126...",0.995684,99\r,17640\r,"new Array(8,7,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,0,0,0\r,0
8,holzgerlingen,Holzgerlingen,17.64 kWp,WR2,"[0, 0, 7, 19, 31, 43, 57, 73, 87, 102, 116, 13...",0.996827,99\r,17640\r,"new Array(8,7,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,0,0,0\r,0
9,hospitalhof,Hospitalhof Stuttgart,36.21 kWp,WR1,[],NaN,99\r,36210\r,"new Array(8,7,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,0\r,0\r,0\r,700\r


In [14]:
temp = pd.read_pickle("app/data/overview.pkl")

In [17]:
temp

,s,Standort,Max. Leistung,Wechselrichter,letzter Tag,Datenqualität,Boot,AnlagenKWP,time_start,time_end,...,DSTMode,Dezimalseparator,WeightUnit,DirectMarketing,AdamAvailable,netProfile,pmControlType,pmReductionOnSerialType,windinverters,co2factor
0,badboll,Ev. Akademie Bad Boll,32.13 kWp,WR1,None,NaN,99\r,32130\r,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,1\r,2\r,0\r,700\r
1,badboll,Ev. Akademie Bad Boll,32.13 kWp,WR2,None,0.967296,99\r,32130\r,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,1\r,2\r,0\r,700\r
2,badboll,Ev. Akademie Bad Boll,32.13 kWp,WR3,None,0.967295,99\r,32130\r,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,1\r,2\r,0\r,700\r
3,esslingen,Esslingen Waldheim,45.0 kWp,WR1,None,NaN,99,45000,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)","new Array(17,18,20,21,21,22,22,21,20,19,17,17)",...,"""1""",""",""","""KG""",false,0,0,0,0,0,460
4,esslingen,Esslingen Waldheim,45.0 kWp,WR2,None,NaN,99,45000,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)","new Array(17,18,20,21,21,22,22,21,20,19,17,17)",...,"""1""",""",""","""KG""",false,0,0,0,0,0,460
5,esslingen,Esslingen Waldheim,45.0 kWp,WR3,None,NaN,99,45000,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)","new Array(17,18,20,21,21,22,22,21,20,19,17,17)",...,"""1""",""",""","""KG""",false,0,0,0,0,0,460
6,geislingen,Ev. Pfarrhaus Geislingen,5.7 kWp,WR1,None,NaN,99\r,5700\r,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r",0,0,0,0,0,0,0,0
7,holzgerlingen,Holzgerlingen,17.64 kWp,WR1,None,0.995794,99\r,17640\r,"new Array(8,7,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,0,0,0\r,0
8,holzgerlingen,Holzgerlingen,17.64 kWp,WR2,None,0.996899,99\r,17640\r,"new Array(8,7,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,0,0,0\r,0
9,hospitalhof,Hospitalhof Stuttgart,36.21 kWp,WR1,None,NaN,99\r,36210\r,"new Array(8,7,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,0\r,0\r,0\r,700\r


In [24]:
temp["letzter Tag"] = None
temp.to_pickle("app/data/overview.pkl")

In [25]:
temp[["HPTitel","HPBetreiber","Max. Leistung","HPModul","HPWR","HPInbetrieb","HPAusricht"]]

,HPTitel,HPBetreiber,Max. Leistung,HPModul,HPWR,HPInbetrieb,HPAusricht
0,"""Ãkumenische Energiegenossenschaft eG""\r","""info@oekumenische-energiegenossenschaft.de""\r",32.13 kWp,"""153 x SF210 210W""\r","""3 x SMA SMC 10000 TL""\r","""10.12.2009""\r","""30Â° SÃ¼d aufgestÃ¤ndert""\r"
1,"""Ãkumenische Energiegenossenschaft eG""\r","""info@oekumenische-energiegenossenschaft.de""\r",32.13 kWp,"""153 x SF210 210W""\r","""3 x SMA SMC 10000 TL""\r","""10.12.2009""\r","""30Â° SÃ¼d aufgestÃ¤ndert""\r"
2,"""Ãkumenische Energiegenossenschaft eG""\r","""info@oekumenische-energiegenossenschaft.de""\r",32.13 kWp,"""153 x SF210 210W""\r","""3 x SMA SMC 10000 TL""\r","""10.12.2009""\r","""30Â° SÃ¼d aufgestÃ¤ndert""\r"
3,"""&Ouml;EG ES SUN 2""","""&Ouml;EG""",45.0 kWp,"""SunPower 225Wp""","""Fronius IG plus 150 / 2 x Plus 70""","""September 2010""","""S&Uuml;D WEST"""
4,"""&Ouml;EG ES SUN 2""","""&Ouml;EG""",45.0 kWp,"""SunPower 225Wp""","""Fronius IG plus 150 / 2 x Plus 70""","""September 2010""","""S&Uuml;D WEST"""
5,"""&Ouml;EG ES SUN 2""","""&Ouml;EG""",45.0 kWp,"""SunPower 225Wp""","""Fronius IG plus 150 / 2 x Plus 70""","""September 2010""","""S&Uuml;D WEST"""
6,"""ÖEG SUN GP 3""\r","""ÖEG""\r",5.7 kWp,"""Schüco MPE 190 MS06""\r","""SMA SMC 5000a""\r","""Mai 2011""\r","""30° Süd-West""\r"
7,"""Johanneskirche Holzgerlingen""\r","""Ökumenische Energiegenossenschaft""\r",17.64 kWp,"""Bosch M60-245""\r","""2 x Kostal PIKO 8.3""\r","""April 2012""\r","""Süd""\r"
8,"""Johanneskirche Holzgerlingen""\r","""Ökumenische Energiegenossenschaft""\r",17.64 kWp,"""Bosch M60-245""\r","""2 x Kostal PIKO 8.3""\r","""April 2012""\r","""Süd""\r"
9,"""Hospitalhof Stuttgart""\r","""Oekumenische Energiegenossenschaft""\r",36.21 kWp,"""REC 255W poly""\r","""2x SMA Tripower 17000""\r","""25.06.2014""\r","""Süden""\r"


In [1]:
import streamlit as st
import pandas as pd
from utils import OverviewDatenManager



st.title("Unsere Solaranlagen")
manager = OverviewDatenManager(standorte=[
    "badboll", "esslingen", "geislingen", "holzgerlingen", "hospitalhof",
    "karlsruhe", "mettingen", "muensingen", "tuebingen", "waiblingen"
])

data = manager.get_dataframe()

2025-11-15 16:31:38.393 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-15 16:31:38.475 
  command:

    streamlit run /Users/simon/Desktop/OEEG Dashboard/.venv/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-11-15 16:31:38.476 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-15 16:31:38.476 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
data

,s,Standort,Max. Leistung,Wechselrichter,letzter Tag,Datenqualität,Boot,AnlagenKWP,time_start,time_end,...,DSTMode,Dezimalseparator,WeightUnit,DirectMarketing,AdamAvailable,netProfile,pmControlType,pmReductionOnSerialType,windinverters,co2factor
0,badboll,Ev. Akademie Bad Boll,32.13 kWp,WR1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,99\r,32130\r,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,1\r,2\r,0\r,700\r
1,badboll,Ev. Akademie Bad Boll,32.13 kWp,WR2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.862425,99\r,32130\r,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,1\r,2\r,0\r,700\r
2,badboll,Ev. Akademie Bad Boll,32.13 kWp,WR3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.862425,99\r,32130\r,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,1\r,2\r,0\r,700\r
3,esslingen,Esslingen Waldheim,45.0 kWp,WR1,[],NaN,99,45000,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)","new Array(17,18,20,21,21,22,22,21,20,19,17,17)",...,"""1""",""",""","""KG""",false,0,0,0,0,0,460
4,esslingen,Esslingen Waldheim,45.0 kWp,WR2,[],NaN,99,45000,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)","new Array(17,18,20,21,21,22,22,21,20,19,17,17)",...,"""1""",""",""","""KG""",false,0,0,0,0,0,460
5,esslingen,Esslingen Waldheim,45.0 kWp,WR3,[],NaN,99,45000,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)","new Array(17,18,20,21,21,22,22,21,20,19,17,17)",...,"""1""",""",""","""KG""",false,0,0,0,0,0,460
6,geislingen,Ev. Pfarrhaus Geislingen,5.7 kWp,WR1,[],NaN,99\r,5700\r,"new Array(8,8,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r",0,0,0,0,0,0,0,0
7,holzgerlingen,Holzgerlingen,17.64 kWp,WR1,[],0.976610,99\r,17640\r,"new Array(8,7,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,0,0,0\r,0
8,holzgerlingen,Holzgerlingen,17.64 kWp,WR2,[],0.982824,99\r,17640\r,"new Array(8,7,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,0,0,0\r,0
9,hospitalhof,Hospitalhof Stuttgart,36.21 kWp,WR1,[],NaN,99\r,36210\r,"new Array(8,7,6,6,6,6,6,7,7,7,7,8)\r","new Array(17,18,20,21,21,22,22,21,20,19,17,17)\r",...,"""1""\r",""",""\r","""KG""\r",false\r,0\r,0\r,0\r,0\r,0\r,700\r
